# Loan Application Orchestration Pipeline (Agent Orchestration)

This notebook demonstrates the full loan application decision process **using the agent orchestration pipeline**. The orchestration function calls all tools and combines the results into a final summary that will be emailed using Zapier workflow and MCP.

The steps of the pipeline are:
1. Check for fraud.
2. Do an SLA check.
3. Interest rate check.
4. Make a recommendation for the loan application.
5. Trigger an email workflow to send an email of the decision.

##### Load in dependencies


In [9]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)

# Import required modules and orchestration pipeline
from data_model import LoanApplicationJourney
from agents import RunContextWrapper

# Import agent objects
from agents.fraud_agent import fraud_agent
from agents.sla_agent import sla_agent
from agents.recommendation_agent import get_recommendation
from agents.report_agent import report_agent
from agents.interest_rate_agent import interest_rate_agent

# Load Zapier objects
from mcp.zapier_channel import ZapierChannel
import os

ModuleNotFoundError: No module named 'mcp.zapier_channel'

#### Create a test loan application

In [10]:
# Create a test LoanApplicationJourney instance
test_app = LoanApplicationJourney(
    application_id="APP-LOCAL-001",
    submitted_time="2025-06-01T09:00:00",
    reviewed_time="2025-06-01T09:30:00",
    approved_time="2025-06-01T10:00:00",
    rejected_time=None,
    processing_steps={"KYC": 72, "CreditCheck": 50, "FinalApproval": 35},
    flagged_for_fraud=False,
    monthly_income=50000,
    monthly_costs=1000,
    requested_amount=25000,
    monthly_debt=400
)

#### Create a deterministic pipeline

In [11]:
def deterministic_agent_pipeline(test_app):
    """
    Orchestrates the deterministic loan application pipeline:
    1. Runs fraud detection on the application.
    2. Performs SLA compliance check.
    3. Calculates the interest rate.
    4. Generates a recommendation based on the above results.
    5. Prepares a summary report.
    6. Sends the report via Zapier webhook.
    """
    # deterministic orchestration of the loan application process
    fraud_result = fraud_agent.tools[0](RunContextWrapper(test_app))
    sla_result = sla_agent.tools[0](RunContextWrapper(test_app))
    interest_rate_result = interest_rate_agent.tools[0](RunContextWrapper(test_app))

    # Get the recommendation based on the results of the previous steps
    recommendation = get_recommendation(fraud_result, sla_result, interest_rate_result)
    print("Recommendation:", recommendation)

    # Prepare the context for the report agent
    final_context = {
        "recommendation": recommendation,
        "fraud_result": fraud_result,
        "sla_result": sla_result,
        "interest_rate_result": interest_rate_result
    }

    # Generate the summary report
    report_result = report_agent.tools[0](RunContextWrapper(final_context))
    print("System Summary Report:\n", report_result)
    # Set your Zapier webhook URL (from your environment or hardcoded for demo)
    zapier_url = os.getenv("MCP_SERVER_URL_SSE", "https://hooks.zapier.com/hooks/catch/your_zapier_id/your_path/")

    zapier_channel = ZapierChannel(webhook_url=zapier_url)

    # Send the report via Zapier
    zapier_channel.send(
        message=report_result,
        metadata={"subject": "Loan Application Recommendation", "to": "anadaria.zahaleanu@email.com"}
    )

    return print("✅ Report sent via Zapier email webhook.")


#### Run the pipeline

In [12]:
deterministic_agent_pipeline(test_app)

/Users/lucadicarlo/Documents/Projects/Oxford-Summit-Loan-Application/agents/interest_rate_agent.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latest_rate = hist['Close'][-1]


Recommendation: **Recommendation: Reject the Loan Application**

While the fraud risk assessment indicates no indicators of fraud and the interest rate result is excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC compliance issues are resolved.
System Summary Report:
 📋 Loan Application System Summary
Dear Daria Zahaleanu,
Recommendation: **Recommendation: Reject the Loan Application**

While the fraud risk assessment indicates no indicators of fraud and the interest rate result is excellent, the SLA compliance has been violated due to issues in the KYC process. This violation raises concerns about the thoroughness of the application review. Therefore, it is advisable to reject the loan application until the KYC compliance issues are resolved.
Fraud Result: {'flagged': False, 'label': 'No fraud indicators

NameError: name 'os' is not defined